# Constructive Solid Geometry (CSG)
These geometries are bases on primitives (e.g. sphere, cylinder, plane) which are used to build solids by performing boolean operations.

Netgen offers the following primitives

| primitive  |  csg syntax |   meaning   |
|:-----------|:------------|:------------|
| half-space | Plane(Pnt a,Vec n)  |     point p in plane, normal vector    
| sphere     | Sphere(Pnt c,float r)|    sphere with center c and radius r 
| cylinder   | Cylinder(Pnt(x<sub>1</sub> a, Pnt b, float r) |     points a and b define the axes of a infinite cylinder of radius r 
| brick      | OrthoBrick ( Pnt a, Pnt b ) |      axes parallel brick with minimal coordinates a and maximal coordinates b

and the boolean operators

| operator  |  set operation |
|:----------|:---------------|
| * 	| intersection
| + 	| union
| - 	| intersection with complement

In [1]:
import netgen.gui
%gui tk

In [2]:
from netgen.csg import *

left  = Plane (Pnt(0,0,0), Vec(-1,0,0) )
right = Plane (Pnt(1,1,1), Vec( 1,0,0) )
front = Plane (Pnt(0,0,0), Vec(0,-1,0) )
back  = Plane (Pnt(1,1,1), Vec(0, 1,0) )
bot   = Plane (Pnt(0,0,0), Vec(0,0,-1) )
top   = Plane (Pnt(1,1,1), Vec(0,0, 1) )

cube = left * right * front * back * bot * top
geo = CSGeometry()
geo.Add (cube)

mesh = geo.GenerateMesh(maxh=0.1)
# mesh.Save("cube.vol")

In [3]:
from netgen.csg import *

cube = OrthoBrick( Pnt(0,0,0), Pnt(1,1,1) )
hole = Cylinder ( Pnt(0.5, 0.5, 0), Pnt(0.5, 0.5, 1), 0.2)

geo = CSGeometry()
geo.Add (cube-hole.maxh(0.05))
mesh = geo.GenerateMesh(maxh=0.1)
# mesh.Save("cube_hole.vol")

## Set properties of solids
A solid has members which we can set to define the desired properties.

In [4]:
sphere = Sphere(Pnt(0,0,0),1)

Now we can set a boundray name on the surface of this sphere

In [5]:
sphere.bc("sphere")

Define a material

In [6]:
sphere.mat("iron")

and a maximal mesh size on the surface

In [7]:
sphere.maxh(0.5)

In case we want to visualize the geometry we can define the color (using rgb values) and transparency oft the solid.

In [10]:
sphere.col([1,0,0])#.transp()

In [11]:
geo = CSGeometry()
geo.Add(sphere)
geo.Draw()

In [12]:
mesh=geo.GenerateMesh(maxh=0.1)